In [1]:
def circular_permuted(x):
    return([x[i:]+x[:i] for i in range(len(x))])

In [2]:
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

def self_and_rev_complement(in_dna):
    all_possible = [in_dna]
    
    # Get reverse complement
    dna = Seq(in_dna, generic_dna)
    rev_complement = str(dna.reverse_complement())
    all_possible.append(rev_complement)
    return(all_possible)

In [3]:
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

def normalise_str(in_dna):
    """Find all possible eqivalent STR sequences. 
    And return the first alphabetically.
    
    For example, TA = AT. But would return AT.
    """
    all_possible = []
    # Circularly permute original sequence and reverse complement
    for seq in self_and_rev_complement(in_dna):
        for permuted_seq in circular_permuted(seq): # Switch to faster permutation (6)
            all_possible.append(permuted_seq)

    # Sort and take the first
    all_possible.sort()
    return(all_possible[0])

In [4]:
# parse VCF files of STR genotype calls (from LobSTR and RepeatSeq)
# Compare the results

import vcf
import pandas as pd
import csv

def parse_vcf(fname, dirname, outname):
    fieldnames=['chr', 'pos', 'refallelelen', 'repeatunit', 'normrepeatunit', 'repeatunitlen', 'genotype', 'depth']
    with open(dirname + fname, 'r') as f:
        with open(outname, 'w') as csvfile:
            csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames, 
                                   delimiter=',', extrasaction='ignore')
            csvwriter.writeheader()
    
            vcf_reader = vcf.Reader(f)
            sample1 = vcf_reader.samples[0] # Get list of samples in the VCF. save the first

            df = pd.DataFrame()
            for record in vcf_reader:
    
                info = {}
        
                # Data to extract:
                # Allele Length Offset(s)" - need to figure out what this is!
        
                # Locus data
                info['chr'] = record.CHROM
                info['pos'] = record.POS
                info['refallelelen'] = record.INFO['RL']
                info['repeatunit'] = record.INFO['RU']
                # Calculated
                info['normrepeatunit'] = normalise_str(info['repeatunit'])
                info['repeatunitlen'] = len(info['repeatunit']) 
                
                # Genotype data
                info['genotype'] = record.genotype(sample1)['GT']

                # Not sure if this is for the individual sample, or over all samples.
                try:
                    info['depth'] = record.INFO['DP']
                except KeyError:
                    try:
                        info['depth'] = record.genotype(sample1)['DP']
                    except KeyError:
                        info['depth'] = None

                csvwriter.writerow(info)        

In [5]:
dirname = '/Users/hd_vlsci/Documents/git/STR-pipelines/data/intersections_LobSTR-RepeatSeq/'
lobstr = 'intersection0_10.vcf' # Called by LobSTR only
both = 'intersection0_10_1.vcf' # Called by LobSTR and RepeatSeq
repeatseq = 'intersection0_11.vcf' # Called by RepeatSeq only

parse_vcf(both, dirname, outname='LobSTR_RepeatSEQ_intersect.csv')
parse_vcf(lobstr, dirname, outname='LobSTR_only.csv')
parse_vcf(repeatseq, dirname, outname='RepeatSEQ_only.csv')